In [1]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits as pyfits
#%matplotlib qt 
%matplotlib inline
from astropy.io import fits as pyfits
from scipy import optimize
from scipy import fftpack
import os  #para programas que fazem iterações em files
import glob
import pandas
import numba
from numba import jit
np.seterr(all='ignore')
from lmfit.models import ExpressionModel

Nota: Este trabalho está feito usando os diretórios do meu computador

#### Passo 1 - Determinação da largura equivalente de riscas fotosféricas

In [2]:
def sigma(λmax, λmin, Res):
    'Calcula sigma'
    return (((λmax-λmin)/2 + λmin) / Res) / (2*np.sqrt(2*np.log(2)))

λmax1 = 5050
λmin1 = 5000  #valores escolhidos para teste
Res1 = 50000  #resolução espetral dada no enunciado
delta_x = 0.004
def gaussiana(x,xc,sigma,Imax,yc):  #d é ycentral
    ''' Função para gerar gaussiana dados 6 parâmetros:
intervalo (x), xcentro (xc), sigma, Intensidade max (Imax) e
ycontínuo (yc)'''
    return Imax * np.exp( - (x - xc) ** 2 / (2 * sigma ** 2)) + yc
def normalize(G):
    return G/G.sum()
def normalize2(G): 
    'divide pelo máximo em vez de dividir pela soma'
    G_max = np.max(G)
    for i in range(len(G)):
        G[i] = G[i]/G_max
    return G

In [3]:
'TESTE LARGURA EQUIVALENTE, usando wave e flux, que vêm da estrela HD17051'
def sigma2(λ_central, Res):
    'Calcula sigma, dado um lambda central'
    d_l = 0.2
    λmax, λmin = λ_central + d_l , λ_central - d_l
    return (((λmax-λmin)/2 + λmin) / Res) / (2*np.sqrt(2*np.log(2)))
d_l = 0.2  #d_lambda


hdulistHD=pyfits.open('C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/HD17051_rv.fits')
#este acima é o observado
hduHD=hdulistHD[0]
headerHD=hduHD.header
fluxHD=hduHD.data
lambda0HD=headerHD["crval1"]
dxHD=headerHD["cdelt1"]
waveHD=np.arange(fluxHD.size)*dxHD+lambda0HD #x
hdulist1=pyfits.open("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/p6250-g+5.0-m0.0-t01-z+0.75-a+0.00.GES4750.fits")
#este acima é p6250(...), ou seja, y
hdulist2=pyfits.open("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/GES_UVESRed580_Lambda.fits")
#este acima é  GES(...), ou seja, x
sintetico1=hdulist1[0].data
sintetico2=hdulist2[0].data
def risca(Lambda_min, Lambda_max, x, espetro):
    #plt.plot(x, espetro)
    #plt.axis([Lambda_min, Lambda_max, Imin, Imax])
    #plt.show()

    lambda_risca=x[(x>Lambda_min)&(x<Lambda_max)]
    espetrocorte=espetro[(x>Lambda_min)&(x<Lambda_max)]
    return lambda_risca, espetrocorte

lambda_risca1, espetrocorte1 = risca(4661.34, 4661.74, waveHD, fluxHD) #usando 4661.54
#print(lambda_risca)

plt.plot(lambda_risca1, espetrocorte1, label = 'risca')


parametros, covar = sp.optimize.curve_fit(gaussiana, lambda_risca1, 
        espetrocorte1, p0 = [4661.54, sigma2(4661.54,50000), -1, 0],  method='lm'  )
print(*parametros)

Gauss = gaussiana(lambda_risca1, *parametros)

plt.plot(lambda_risca1, Gauss, label = 'ajuste gauss.')
plt.legend()
plt.show()
#yc, Imax e sigma
p1 = -parametros[2] #Imax
p2 = parametros[3]  #yc (presumindo que está no zero e anda para baixo, isto é zero)
p3 = parametros[1]  #sigma
Equiv_Width = np.sqrt(2 * np.pi)*(p1 / p2 )* p3 *1e3  #por ser mA
print(Equiv_Width)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/HD17051_rv.fits'

#### Passo 2 - Aplicação do código anterior para determinar Wλ na estrela em análise (Sol)

Célula que elimina riscas FeII

In [ ]:
lista_tsantaki = np.loadtxt("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/line_list_tsantaki.dat", usecols = (0,1,2,4))


lista_tsantaki2 = lista_tsantaki #para deixar a variável lista_tsantaki intacta
lista_val = [0,1,2,3]
print('Len é' , len(lista_tsantaki2))
#print(lista_tsantaki2[128,0])
vari = 0
#vari2 = 0
array_tsantaki = np.zeros((120,4))
#while len(lista_tsantaki3) < 120:
while vari < 120:
    vari2 = 0
    while vari2 < 4:
    #for k in range(0,4):   
       # print(vari)
        #lista_tsantaki2 = np.delete(lista_tsantaki2,-vari,0)
        array_tsantaki[vari,vari2] = lista_tsantaki[vari,vari2]
        vari2 += 1
    vari += 1
print(len(array_tsantaki))

In [ ]:
#Larguras equivalentes para array_tsantaki
lambdas=array_tsantaki[:,0]   #para usar só riscas de FeI, uso array_tsantaki

def risca(lambda_min, lambda_max, x, espetro):  
    lambda_risca=x[(x>lambda_min)&(x<lambda_max)]
    espetro_risca=espetro[(x>lambda_min)&(x<lambda_max)]
    return lambda_risca, espetro_risca
   
def larg_equiv_tsantaki():    
    LE=[]

    for i in lambdas:
        lambda_min=i-d_l
        lambda_max=i+d_l     
        ilambda_risca, iespetro_risca = risca(lambda_min, lambda_max, waveHD, fluxHD)
        iparam_, inutil = sp.optimize.curve_fit(gaussiana, ilambda_risca,
                                iespetro_risca, [i, sigma2(i,Res1), -1, 0] )

        print(iparam_)
        plt.plot(ilambda_risca, iespetro_risca, label = 'risca')
        GG = gaussiana(ilambda_risca, *iparam_)  #este * torna em array automaticamente
        plt.plot(ilambda_risca, GG, label = 'ajuste gauss.')
        plt.legend()
        plt.show()

        a = - iparam_[2]  #Imax; vem negativo mas queremos positivo
        b = iparam_[3] # yc
        c = iparam_[1]  #sigma
        Wh = np.sqrt(2*np.pi)*(a/b)*c * 1e3 #Por ser mA
        LE.append(Wh)

    print(LE)
    return LE
#Ajusta-se bem nas riscas "bem comportadas"
larg_equiv_tsantaki()

#### Passo 3 - Estimativa da temperatura de excitação utilizando a curva de crescimento

 multipletos escolhidos: 
 
 multipleto 1 - [3.6 , 3.8];
 
  multipleto 6+7 - [4.5, 4.7]
                    
Comentário acerca da notação: Vou chamar a este segundo multipleto "Multipleto 6+7" e não "2º Multipleto" ou "Multipleto 2", achando mais conveniente considerar o número global de multipletos da estrela, como indicados nos slides do professor.
                         
ou seja,  $\chi_{1}$  ∈ [3.6,3.8] e $\chi_{2}$ ∈ [4.5,4.7].


Abaixo é programa para adicionar elementos aos multipletos


In [ ]:
mult1_sol = np.zeros((12,4))  #descobri número de linhas do necessárias na tabela (ou 
#seja, o número de riscas neste multipleto) simplesmente correndo o código abaixo e 
#vendo a partir de que linha começava a ser tudo zero.
vari3 = 0
mult2_sol = np.zeros((26,4))
vari4 = 0

print(len(array_tsantaki))
for j in range(len(array_tsantaki)):
    #vai a cada linha, na mesma coluna
    #print('j   ', j)
    #print(array_tsantaki[j,1])
    if 3.6 <= array_tsantaki[j,1] <= 3.8:
        #if array_tsantaki[j,1] > 3.6:
            print('ENCONTRADO ELEMENTO multipleto1: posição ', j,'   EP =',  array_tsantaki[j,1])
            mult1_sol[vari3,:] = array_tsantaki[j,:]
            vari3+=1
    elif 4.5 <= array_tsantaki[j,1] <= 4.7:
        #if array_tsantaki[j,1] < 4.7:
            print('ENCONTRADO ELEMENTO multipleto 6+7: posição ', j,'   EP =',  array_tsantaki[j,1])
            mult2_sol[vari4,:] = array_tsantaki[j,:]
            vari4 +=1
            #print(vari4) 
print('MULTI 1 = ', mult1_sol, '\n Número de riscas do multipleto 1 = ', len(mult1_sol))
print('MULTI 2 = ', mult2_sol, '\n Número de riscas do multipleto 6+7 = ', len(mult2_sol))


In [ ]:
def larg_equiv_sol(): #DE ONDE SE CALCULA A TEMPERATURA TAMBÉM
# multipleto 1 - [3.6 , 3.8]   - Valor médio = (EP_max - EP_min) / 2 = 3.7
# multipleto 6+7 - [4.5, 4.7]  - Valor médio = 4.6
   
    plt.scatter((mult1_sol[:,2]+np.log(mult1_sol[:,0])),(np.log(mult1_sol[:,3]/(mult1_sol[:,0]))))
    plt.scatter((mult2_sol[:,2]+np.log(mult2_sol[:,0])),(np.log(mult2_sol[:,3]/mult2_sol[:,0])))
    
        
    plt.scatter((mult1_sol[:,2]+np.log(mult1_sol[:,0])),(np.log(mult1_sol[:,3]/(mult1_sol[:,0]))))
    plt.scatter((mult2_sol[:,2]+np.log(mult2_sol[:,0])),(np.log(mult2_sol[:,3]/mult2_sol[:,0])))
    
    
    'CENTROIDE-mult1_sol'
    centroidx1=np.average(mult1_sol[:,2]+np.log(mult1_sol[:,0]))
    centroidy1=np.average(np.log(mult1_sol[:,3]/(mult1_sol[:,0])))
    'CENTROIDE-mult2_sol'
    centroidx2=np.average(mult2_sol[:,2]+np.log(mult2_sol[:,0]))
    centroidy2=np.average(np.log(mult2_sol[:,3]/mult2_sol[:,0]))
    
    plt.scatter(centroidx1,centroidy1, c = 'fuchsia', label = 'centróide mult.1')
    plt.scatter(centroidx2,centroidy2, c = 'aqua', label = 'centróide mult.2') #marcar os 2 centroides no scatter
    
    delta=centroidx2-centroidx1
    print('A largura equivalente é ' + str(delta))
    
    T_exc=abs(5040*(4.6-3.7)/delta) #diferença dos valores médios
    #erro=abs(5778-T_exc)/5778*100
    #print(erro)
    
    print('A temperatura é ' + str(T_exc), ' K')
    print('Erro relativo (comparação com T(Sol) = 5780K) = ' , ((abs(T_exc - 5780)/5780)) * 100, ' %')
    plt.legend(loc='lower right')
    plt.show()
    return T_exc
    
larg_equiv_sol()

#### Passo 4 : Por comparação com a grelha de espetros sintéticos (grelha acessível no Moodle em library.tgz), determinar o espetro sintético cujas Wλ melhor aproxime as obtidas na estrela em análise.

In [ ]:
T_exc = larg_equiv_sol()
T_eff = T_exc
#Tomamos todas as metalicidades e índices químicos

aceites = []  #aceites é array com nomes das files aceites, ou seja, que cumprem
#os requisitos de T e log(g)

for filename in os.listdir("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/library/Zipado"):
    if T_eff - 400 < float(filename[1:5]) < T_eff + 400:  #condição T
        if float(filename[7:11]) >= 3.5:    #condição log(g)
            aceites.append(filename)
        
for i in range(len(aceites)):
    print('  File nº ', i, 'é ', aceites[i])
    

In [ ]:
def risca_def(lambda_min,lambda_max,x,espetro):
    #corta o espetro
    lambda_risca1 = []
    espetro_risca1 = []
    for i in range(len(x)):
        #print(i)
        if lambda_min < x[i] < lambda_max:
            lambda_risca1.append(x[i])
            espetro_risca1.append(espetro[i])
    return lambda_risca1, espetro_risca1

def img_obs(dir_fich):                          
     
    hdulist = pyfits.open(dir_fich)            
    hdu = hdulist[0]
    header = hdu.header
    fluxo_observado = hdu.data        
    lambda0 = header["crval1"]
    dx = header["cdelt1"]
    wave_obs = sp.arange(fluxo_observado.size)*dx + lambda0  
    #flux_norm = normalize2(fluxo_observado)    
        
    return wave_obs, fluxo_observado

def img_sint(dir_lamb,dir_flux):
    
    hdulist1 = pyfits.open(dir_lamb)            
    hdu1 = hdulist1[0]
    header1 = hdu1.header
    wave_sint = hdu1.data
    hdulist2 = pyfits.open(dir_flux)                                         
    hdu2 = hdulist2[0]
    header2 = hdu2.header
    flux_sint = hdu2.data
    #flux_norm = normalize2(flux_sint)
    return wave_sint, flux_sint


In [ ]:
'Esta célula é teste - alterar dir_fluxo_observado para diretório desejado'
dir_fluxo_observado = "C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/estrelas_a_analisar/estrela2_vcor.fits"

wave_len_obs, fluxo_observado  =img_obs(dir_fluxo_observado)

#CONSIDERAR APENAS A SEÇÃO DE LAMBDAS 5900-6800

file=np.loadtxt("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/line_list_tsantaki.dat", usecols=(0,1,2,4))
Lambdas, EP = file[:,0], file[:,1]
Lambs = Lambdas[(Lambdas>5900 )&(Lambdas<6800)]
EP_1 = file[(Lambdas>5900 )&(Lambdas<6800)]
ep_1 = EP_1[:,1]

#wavelen dos sinteticos
hdulist2=pyfits.open("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/GES_UVESRed580_Lambda.fits")
wave_len_sinteticos =hdulist2[0].data


'NORMALIZAÇÃO DO ESPETRO OBSERVADO'
Lmin = 5900
Lmax = 6800
def norm_espetro_obs(L1, L2, espetro):
    
    wave_len_obs, fluxo_observado = img_obs(espetro)
    #print(wave_len_obs)
    wave  = wave_len_obs[(wave_len_obs>Lmin)&(wave_len_obs<Lmax)]
    flx   = fluxo_observado[(wave_len_obs>Lmin)&(wave_len_obs<Lmax)]
    fluxo = flx/max(flx)   
    return wave, fluxo


wave, fluxo = norm_espetro_obs(Lmin,Lmax,dir_fluxo_observado)
plt.plot(wave,fluxo)
plt.xlim(Lmin,Lmax)
plt.show()

In [ ]:
def risca_cent(LambCtrl, wave_len, flux):
    Lmin=LambCtrl-0.2
    Lmax=LambCtrl+0.2
    
    xcorte=wave_len[(wave_len >Lmin)&(wave_len <Lmax)]
    #print(xcorte)
    espectrocorte=flux[(wave_len >Lmin)&(wave_len <Lmax)]
    #print(espectrocorte)
    
    return xcorte, espectrocorte
def sigma3(λCentral, RES):
    λmax=λCentral+0.2
    λmin=λCentral-0.2
    dλ = ((λmax- λmin)/2 + λmin) / RES
    c = dλ / (2*np.sqrt(2*np.log(2)))
    return c, dλ

In [ ]:
def larg_equiv_ger(espetro,lambd): 
        #este lambd é lambd onde queremos calcular
    d_l = 0.2
    hdu = espetro[0]
    header = hdu.header
    flux1 = hdu.data
    lambda0=header["crval1"]
    dx=header["cdelt1"]
    wave1 = np.arange(flux1.size)*dx+lambda0
    LE=[]

    for i in lambd:
        lambda_min=i-d_l
        lambda_max=i+d_l     
        ilambda_risca, iespetro_risca = risca_def(lambda_min, lambda_max, wave1, flux1)
        iparam_, inutil = sp.optimize.curve_fit(gaussiana, ilambda_risca, iespetro_risca, [i, sigma(i-d_l,i+d_l,Res1), -1, 0] )

        #print(iparam_)
        #plt.plot(ilambda_risca, iespetro_risca, label = 'risca')
        GG = gaussiana(ilambda_risca, *iparam_)  #este * torna em array automaticamente
        #plt.plot(ilambda_risca, GG, 'indigo', label = 'ajuste gauss.')
       # plt.legend()
       # plt.show()

        a = - iparam_[2]  #ele vem negativo, daí o sinal -
        b = iparam_[3]
        c = iparam_[1]
        Wh = np.sqrt(2*np.pi)*(a/b)*c * 1e3
        LE.append(Wh)
    #print(np.array(LE))
    return np.array(LE)
#Testar o código
espetro_est1 =pyfits.open("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/estrelas_a_analisar/estrela1.fits")
LE = larg_equiv_ger(espetro_est1, Lambs)
print(LE)

In [ ]:
RES = Res1 
print(len(aceites))
minimo = np.min(wave_len_sinteticos)
rangee = Lambs[Lambs>minimo]

LE1 =(LE[Lambs>minimo])
VAL_Erros=[]
var_contagem = 0
for j in aceites:
    print(var_contagem)
    ERROS=[]
    LEE=[]
    
    wave_s, flux_s = img_sint("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/GES_UVESRed580_Lambda.fits","C:/Users/bernc/Desktop/library/Zipado/" + str(j))
    
    
    for i in rangee:  
        ixcorte_s, iespectrocorte_s = risca_cent(i, wave_s, flux_s)
        
        if len(ixcorte_s) !=0 or len(iespectrocorte_s) !=0 :
            iparam_s, inutil = sp.optimize.curve_fit(gaussiana, ixcorte_s, iespectrocorte_s, [-1,i,sigma3(i,RES)[0],1], bounds=((-1,i-0.1,sigma3(i,RES)[0]-0.1,0.9),(0,i+0.1,sigma3(i,RES)[0]+0.1,1)) , maxfev=4000)            

                        
            a =-iparam_s[0]
            b = iparam_s[3]
            c = iparam_s[2]
            eq_Wh = np.sqrt(2*np.pi)*(a/b)*c * 1e3
            
            #print(eq_Wh)
            LEE.append(eq_Wh)      
            #print(*iparam_s)
                        
                                
    LEE=np.stack(LEE)
    #print(LEE)  
    
    for e in range(len(LEE)):
        
        ee=(LEE[e]-LE1[e])**2
        ERROS.append((LEE[e]-LE1[e])**2)
            
    #print(ERROS)
    VAL_Erros.append(np.sum(ERROS))
    min(VAL_Erros)
    for l in range(len(VAL_Erros)):
        if VAL_Erros[l] == min(VAL_Erros):
            bb=l
            break
    var_contagem+=1

#print((VAL_Erros[bb]))
print(aceites[bb])

#
'''Este programa demora algum tempo a correr, mas isso deve-se ao facto de estar a fazer
tantos cálculos: está a percorrer 180 ficheiros, a calcular as larguras equivalentes
para cada uma das riscas de cada um e, em seguida, a calcular o erro de cada largura
em relação às observadas.'''

In [ ]:
def comparacao_min_quad(espetro_obs,lambd):
    RES = Res1
    minimo = np.min(wave_len_sinteticos)
    rangee = Lambs[Lambs>minimo]
    LE = larg_equiv_ger(espetro_obs,lambd)
    LE1 =(LE[Lambs>minimo])
    VAL_Erros=[]

    for j in aceites:

        ERROS=[]
        LEE=[]

        wave_s, flux_s = img_sint("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/GES_UVESRed580_Lambda.fits","C:/Users/bernc/Desktop/library/Zipado/" + str(j))


        for i in rangee:        
            ixcorte_s, iespectrocorte_s = risca(i, wave_s, flux_s)

            if len(ixcorte_s) !=0 or len(iespectrocorte_s) !=0 :

                iparam_s, inutil = sp.optimize.curve_fit(gaussiana, ixcorte_s, iespectrocorte_s, [-1,i,sigma(i,RES)[0],1], bounds=((-1,i-0.1,sigma(i,RES)[0]-0.1,0.9),(0,i+0.1,sigma(i,RES)[0]+0.1,1)) , maxfev=4000)            
        #Comentar mudança dos parâmetros iniciais
   

                a =-iparam_s[0]
                b = iparam_s[3]
                c = iparam_s[2]
                eq_Wh = np.sqrt(2*np.pi)*(a/b)*c * 1e3

                #print(eq_Wh)
                LEE.append(eq_Wh)      
                #print(*iparam_s)


        LEE=np.stack(LEE)
        #print(LEE)  

        for e in range(len(LEE)):

            ee=(LEE[e]-LE1[e])**2
            ERROS.append((LEE[e]-LE1[e])**2)

        #print(ERROS)
        VAL_Erros.append(np.sum(ERROS))
        min(VAL_Erros)
        for l in range(len(VAL_Erros)):
            if VAL_Erros[l] == min(VAL_Erros):
                bb=l
                break


    print((VAL_Erros[bb]))
    print(aceites[bb])

In [ ]:
#Aqui ainda sem convolução com função de rotação, que será feita à frente

x,y = img_sint("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/GES_UVESRed580_Lambda.fits","C:/Users/bernc/Desktop/library/Zipado/" +str(aceites[bb]))


L1=6600
L2=L1+10
plt.plot(wave,fluxo, label='Espetro Observado')
plt.plot(x,y/1.05, label='Espetro Sintético')
plt.xlim(L1,L2)
plt.legend()
plt.show()


#### Passo 5: Determinar velocidade de rotação projetada na direção do observador, v * sin(I), para a estrela em análise. Se não conseguir determinar o v * sin(I) pode assumir v * sin(I) = 10 km/s.

In [ ]:


def lim_1(lambcentral,x,espectro):
    lambmax=[]
    lambmin=[]
    cc=[]
    ll=[] 
    sect=[] 
    deriv=np.gradient(espectro,x,edge_order=2) #Derivadas da risca
    for i in lambcentral:
        lam_cent_m=i-0.05
        lamb_cent_alma=i+0.05
        if (len(espectro[(x>lam_cent_m)&(x<lamb_cent_alma)]))==0 : #Não há risca nesta zona
            continue
        arg=(np.argwhere(espectro==np.amin(espectro[(x>lam_cent_m)&(x<lamb_cent_alma)]))) #argumento onde está o máximo da risca
        lambcentralr=float(x[arg[0]]) #Máximo da risca
        a=1 #Iteração para o lado direito
        b=1 #Iteração para o lado esquerdo
        c=np.argwhere(x==lambcentralr) #Argumento do lambcentral
        if np.sign(deriv[c])==np.sign(deriv[c+1]): #braço direito é do mesmo sinal
            while (deriv[c]*deriv[(c+a)])>0:
                a+=1
            while (deriv[c]*deriv[c-b])<0:
                b+=1
        else:
            while (deriv[c]*deriv[c+a])<0: #Braço direito é de sinal contrario ao do máximo
                a+=1
            while (deriv[c]*deriv[c-b])>0:
                b+=1
        if a <10: #Se não passar a condição, a risca é fortemente deformada no braço direito
            sect.append(i)
        else:
            if b<10: #Se não passar a condição, a risca é fortemente deformada no braço esquerdo
                sect.append(i)
        ll.append(i)
        lambmax=np.append(lambmax,float(x[c+a]))
        lambmin=np.append(lambmin,float(x[c-b]))
    for j in range(len(sect)):
        cc.append(np.argwhere(ll==sect[j]))
    ll=np.delete(ll,cc) #Retirada das riscas inaptas
    lambmin=np.delete(lambmin,cc)
    lambmax=np.delete(lambmax,cc)
    return np.stack((ll,lambmin,lambmax))

In [ ]:
def perf_rotacao(lambcentral,vsin,lamb):
    deltlamb=lambcentral*vsin/300000
    aa=[]
    for i in range(len(lamb)):
        if abs(lamb[i])<deltlamb:
            arg=1-np.power(((lamb[i])/deltlamb),2)
            func=(2*(1-0.6)*(arg**0.5)+(np.pi*0.6/2)*arg)/(np.pi*deltlamb*(1-(0.6/3)))
            aa.append(func)
        else:
            aa.append(0)
    aa=np.array(aa)
    aa=normalize(aa)  #ver se é normalize ou normalize2
    return aa
def func_vsin(x,espectro,data):
    lambcentral=data[0,:]
    lambmin=data[1,:]
    lambmax=data[2,:]
    dx=x[1]-x[0]
    zero=[]
    xcort=x[(x>lambmin[0])&(x<(lambmax[0]+(399*dx)))]
    espetro_corte=espectro[(x>lambmin[0])&(x<lambmax[0])]
    espetro_corte=np.insert((espetro_corte-espetro_corte[-1]),-1,(np.zeros(200)))
    espetro_corte=np.insert((espetro_corte),0,espetro_corte[0]*(np.ones(200)))
    tf=sp.fftpack.fft(espetro_corte+1)
    ps=abs(tf)
    freq=(fftpack.fftfreq(espetro_corte.size,d=dx))
    zero_Fourier=np.argwhere(ps<1e-2)
    for j in range(len(zero_Fourier)-1):
        if abs(zero_Fourier[j]-zero_Fourier[j+1])!=1:
            zero.append(zero_Fourier[j])
            zero.append(zero_Fourier[j+1])
    deltalamb=0.660/freq[np.argwhere(ps==ps[zero[0]])]
    vsin=deltalamb*300000/lambcentral[0]
    return(vsin[0])

In [ ]:
def v_sin_estrela(diret_estrela):
    x_observado,fluxo_observado = img_obs(diret_estrela)
    lambs = lim_1(lambdas, x_observado, fluxo_observado)
    #print('\n LAMBDS: \n' , lambs)
    vsin=func_vsin(x_observado,fluxo_observado,lambs)
    print(vsin)
'estrela1'
a = v_sin_estrela("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/estrelas_a_analisar/estrela1.fits")

In [ ]:
def v_sin_estrelaROT(diret_estrela):
    x_observado,fluxo_observado = img_obs(diret_estrela)
    lambs = lim_1(lambdas, x_observado, fluxo_observado)
    #print('\n LAMBDS: \n' , lambs)
    vsin=func_vsin(x_observado,fluxo_observado,lambs)
    print(vsin)
    x_medio=(6147+6155)/2
    xx=x_medio-np.linspace(6147,6155,1000)
    rot=perf_rotacao(x_medio,int(vsin),xx)
    plt.plot(xx,rot)

f_rot = v_sin_estrelaROT("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/estrelas_a_analisar/estrela1.fits")

In [ ]:
#Nota: Para se conseguir obter a função de rotação e convolução para a estrela 2,
#é preciso limitar o espetro à faixa 6000-6500, o que não tem mal pois, de qualquer
#forma, não queremos estudar a globalidade do espetro neste caso.

#### Passo 6: Para os parâmetros finais obtidos, representar o espetro sintético depois de corrigido do perfil instrumental (assumindo que o espetro da estrela tem uma resolucão espetral igual a 50 000), da velocidade de rotação v * sin(I) e amostrado para a mesma largura de pixel do espetro observado. Comparar o espetro simulado com o espetro observado.

In [ ]:
def simulação_inicial(sintetico,observado, dλ):
    'esta função faz a convolução entre o espetro sintético e o observado '
    convolução=np.convolve(sintetico,gaussiana(x,0,sigma(λmax1,λmin1,Res1),1, 0),mode="same")
    return convolução
def abre_ficheiro(filename):
    hud=pyfits.open(filename)
    return hud[0].data
def simulação(ysint,xsint,xobservado,R): #Convulção e interpolação de um especto sintético
    dx=xobservado[1]-xobservado[0]
    sig=sigma(6155,6147,R) #Intervalo de análise
    xval=np.arange(-5*sig,5*sig+dx,dx) 
    gauss=gaussiana(xval,sig,0,1,0)
    gauss=gauss/gauss.sum() #normalização da gaussiana
    convulsão=np.convolve(ysint,gauss,mode="same")
    interp=np.interp(xobservado,xsint,convulsão)
    return interp

In [ ]:
def conv_fin(diret_estrela_obs):
    x_observado,fluxo_observado = img_obs(diret_estrela_obs)
    x_sintetico=abre_ficheiro("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/GES_UVESRed580_Lambda.fits")
    #print('x_sintetico', x_sintetico)
    file=np.loadtxt("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/line_list_tsantaki.dat",usecols=(0,1,2))
    comp_ond=file[:,0]
    #print('comp_ond', comp_ond)
    lambs=lim_1(comp_ond,x_observado,fluxo_observado)
    #print('lambs', lambs)
    vsin=func_vsin(x_observado,fluxo_observado,lambs)
    #print('vsin', vsin)
    nome_r = "C:/Users/bernc/Desktop/library/Zipado/" + str(aceites[bb])
    print('nome_r  ', nome_r)
    flux_r=abre_ficheiro(nome_r)
    #print(flux_r)
    fluxo_convol=simulação(flux_r,x_sintetico,x_observado,50000)
    print(fluxo_convol)
    x_medio=(6147+6155)/2
    xx=x_medio-np.linspace(6147,6155,1000)
    rot=perf_rotacao(x_medio,int(vsin[0]),xx)
    fluxo_convol=np.convolve(fluxo_convol,rot,mode="same")
    print(fluxo_convol)
    plt.plot(xx,rot) 
    
    plt.show()
    plt.plot(x_observado[(x_observado>6147)&(x_observado<6155)],normalize(fluxo_observado[(x_observado>6147)&(x_observado<6155)]))
    
    plt.plot(x_observado[(x_observado>6147)&(x_observado<6155)],normalize(fluxo_convol[(x_observado>6147)&(x_observado<6155)]))
    plt.show()
conv_fin("C:/Users/bernc/Desktop/3ª ano 1ª semestre/Astronomia Computacional/Trabalho 1 - Determinação de parâmetros estelares/estrelas_a_analisar/estrela1.fits")

Nota: Antes estava a correr direito, mas agora, por algum motivo, a convolução está a dar um array de None. A imagem antes, que por acaso guardei para colocar no relatório, era esta:

https://i.gyazo.com/f8688b84348bb60e2046cadde06a2a33.png

### Apêndice: Função para cálculo automático da temperatura através das curvas de crescimento

In [ ]:
def temp_automatica_curva_crescimento(espetro,lambd):
    d_l = 0.2
    wave_1, flux_1 = img_sint(lambd,espetro)
    LE=[]

    for i in lambd:
        print(i)
        lambda_min=i-d_l
        lambda_max=i+d_l     
        ilambda_risca, iespetro_risca = risca_def(lambda_min, lambda_max, wave1, flux1)
        iparam_, inutil = sp.optimize.curve_fit(gaussiana, ilambda_risca, iespetro_risca, [i, sigma(i-d_l,i+d_l,Res1), -1, 0] )

        #print(iparam_)
        #plt.plot(ilambda_risca, iespetro_risca, label = 'risca')
        GG = gaussiana(ilambda_risca, *iparam_)  #este * torna em array automaticamente
        #plt.plot(ilambda_risca, GG, 'indigo', label = 'ajuste gauss.')
       # plt.legend()
       # plt.show()

        a = - iparam_[2]  #ele vem negativo, daí o sinal -
        b = iparam_[3]
        c = iparam_[1]
        Wh = np.sqrt(2*np.pi)*(a/b)*c * 1e3
        LE.append(Wh)
    LE_def = np.array(LE)
    #print(np.array(LE))
    EP1, EP2, EP3, EP4 = 1.56, 3.88, 4.58, 4.79
    M1=EP_1[(ep_1>=EP1)&(ep_1<=EP2)]
    M2=EP_1[(ep_1>=EP3)&(ep_1<=EP4)]

    ml1=LE_def[(ep_1>=EP1)&(ep_1<=EP2)]
    ml2=LE_def[(ep_1>=EP3)&(ep_1<=EP4)]


    plt.scatter((M1[:,2]+np.log(M1[:,0])),(np.log(ml1/(M1[:,0]))), label= 'Multipleto 1')
    plt.scatter((M2[:,2]+np.log(M2[:,0])),(np.log(ml2/M2[:,0])), label= 'Multipleto 1')
    plt.xlabel('log(λgf)')
    plt.ylabel('log($W_λ/λ$)')

    'CENTROIDE-mult1'
    centroidx1=np.average(M1[:,2]+np.log(M1[:,0]))
    centroidy1=np.average(np.log(ml1/(M1[:,0])))

    'CENTROIDE-mult2'
    centroidx2=np.average(M2[:,2]+np.log(M2[:,0]))
    centroidy2=np.average(np.log(ml2/M2[:,0]))

    plt.scatter(centroidx1,centroidy1, color = 'black')
    plt.scatter(centroidx2,centroidy2, color = 'black', label= 'centróides')
    #marcar os 2 centroides no scatter
    plt.legend()

    dist=(((centroidx2-centroidx1)**2)+((centroidy1-centroidy2))**2)**(1/2)

    Texc=abs(5040*((EP1+EP2)/2-(EP3+EP4)/2)/dist)    
    #erro=abs(5800-Texc)/5800*100   #Usar para verificação da estrela 1, cuja temperatura
    #é sabida (5800 K). O erro obtido foi de ~2%, o que é muito bom.


    #print('A temperatura é ' + str(Texc) + ' com um erro de ' + str(erro) + '%')
    print('\n A temperatura é', Texc , ' K')
    return Texc